# Are we consuming more local?

## Research questions

1. Where are the products we consume in our everyday life coming from?

    - Which countries produce the primary resources (ground ingredients) consumed in Switzerland?
    - Which countries manufacture most of the products consumed in Switzerland?


2. Is there a trend over time to consume more local products?

    - Are new products mostly using primary resources from Switzerland? Or from other countries inside Europe?
    - Are new products mostly manufactured in Switzerland? Or from other countries inside Europe?
    - Is there a trend over time to local products to promote their origin?

## Datasets

Open Food Facts (https://world.openfoodfacts.org/data)

Additional datasets “Evolution de la consommation de denrées alimentaires en Suisse” (https://opendata.swiss/fr/dataset/entwicklung-des-nahrungsmittelverbrauches-in-der-schweiz-je-kopf-und-jahr1) and “Dépenses fédérales pour l’agriculture et l’alimentation” (https://opendata.swiss/fr/dataset/bundesausgaben-fur-die-landwirtschaft-und-die-ernahrung1) from https://opendata.swiss/fr/group/agriculture

A last additional dataset for the second question of the project
https://www.gate.ezv.admin.ch/swissimpex/public/bereiche/waren/result.xhtml
Total of imports of agriculture, forestry and fishing goods


In [1]:
import re
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import timedelta
import findspark
findspark.init()
import pyspark

from functools import reduce
from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import to_date, last_day,date_add
from datetime import timedelta

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
DATA_FOLDER = 'data'
openfood_file = "/en.openfoodfacts.org.products.csv"
PARQUET_PATH = 'output/data.parquet'
temp_file = 'output/temp.csv'

# Loading data

In [ ]:
dataset_main = spark.read.csv(DATA_FOLDER+ openfood_file, header=True, mode="DROPMALFORMED", sep = '\t')

dataset_main.createOrReplaceTempView("data_main")

# Filter required columns
p_id_col = " code, product_name, "
general_cols = " brands, brands_tags, categories, categories_tags, origins, origins_tags, manufacturing_places, manufacturing_places_tags,labels,labels_tags,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags "
geo_cols = " origins, manufacturing_places, countries "
geo_tags_cols = " origins_tags, manufacturing_places_tags, countries_tags "

off_df = spark.sql("SELECT" + p_id_col + geo_tags_cols + " FROM data_main")
off_df.printSchema()

### Data Cleaning and Preprocessing

In [ ]:
off_df.createOrReplaceTempView("off_df")

sql_filter = "SELECT * FROM off_df WHERE countries_tags is not NULL AND manufacturing_places_tags is not NULL AND origins_tags is not NULL"
off_p_df = spark.sql(sql_filter)

# off_p_all_size = off_p_df.count()
# off_p_cols_size = len(off_p_df.columns)
# print("Full GEO information data Size:\n" + str(off_p_cols_size) + "(columns) * " + str(off_p_all_size) + "(rows)")

In [ ]:
# Explode countries
off_p_df = off_p_df.withColumn('origins_tags', F.explode_outer(F.split('origins_tags', ',')))\
.withColumn('manufacturing_places_tags', F.explode_outer(F.split('manufacturing_places_tags', ',')))\
.withColumn('countries_tags', F.explode_outer(F.split('countries_tags', ',')))

# Remove "en:" occurances before name of each country in coutries_tags
off_p_df = off_p_df.withColumn('countries_tags', F.regexp_replace('countries_tags', "en:", ""))
# Remove - in name of countries
off_p_df = off_p_df.withColumn('countries_tags', F.regexp_replace('countries_tags', "-", " "))
off_p_df = off_p_df.withColumn('manufacturing_places_tags', F.regexp_replace('manufacturing_places_tags', "-", " "))
off_p_df = off_p_df.withColumn('origins_tags', F.regexp_replace('origins_tags', "-", " "))

# Remove numbers from name of countries
off_p_df = off_p_df.withColumn('countries_tags', F.regexp_replace('countries_tags', "\d+", ""))
off_p_df = off_p_df.withColumn('manufacturing_places_tags', F.regexp_replace('countries_tags', "\d+", ""))
off_p_df = off_p_df.withColumn('origins_tags', F.regexp_replace('origins_tags', "\d+", ""))

In [ ]:
off_p_df = off_p_df.withColumn("countries", off_p_df["countries_tags"])
off_p_df = off_p_df.withColumn("manufacturing_places", off_p_df["manufacturing_places_tags"])
off_p_df = off_p_df.withColumn("origins", off_p_df["origins_tags"])
off_p_df = off_p_df.withColumn("is_eu", lit(0))
off_p_df.printSchema()

In [ ]:
off_p_all_size = off_p_df.count()
off_p_cols_size = len(off_p_df.columns)
print("Full GEO information data Size:\n" + str(off_p_cols_size) + "(columns) * " + str(off_p_all_size) + "(rows)")

In [ ]:
# df['c_countries'] = ""
# df['c_manufacturing_places'] = ""
# df['c_origins'] = ""
# df['sold_in_EU'] = 0

# for index, row in df.iterrows():
#     output = cleaned_countries[cleaned_countries.input.str.contains(row.origins_tags, case=False, na=False)]
#     if len(output):
#         row.c_origins = output.iloc[0].country_code
        
#     output = cleaned_countries[cleaned_countries.input.str.contains(row.manufacturing_places_tags, case=False, na=False)]
#     if len(output):
#         row.c_manufacturing_places = output.iloc[0].country_code
        
#     output = cleaned_countries[cleaned_countries.input.str.contains(row.countries_tags, case=False, na=False)]
#     if len(output):
#         row.c_countries = output.iloc[0].country_code
        
#     output = eu_countries[eu_countries.str.contains(row.c_countries, case=False, na=False)]
#     if len(output):
#         row.sold_in_EU = 1

In [4]:
# Clean countries
cleaned_countries= pd.read_csv(DATA_FOLDER+ "/mapping_countries.csv",sep=',', error_bad_lines=False, encoding = "utf-8")
cleaned_countries['country_code'] = cleaned_countries['country_code'].str.lower()
cleaned_countries.head()

,Unnamed: 0,input,country_code
0,0,france,fr
1,1,united kingdom,uk
2,2,canada,ca
3,3,germany,gm
4,4,vietnam,vm


In [3]:
seen = 4500

In [10]:
interval = 501
if seen:
#     off_p_df = spark.read.parquet(PARQUET_PATH)
    off_p_df = spark.read.csv(temp_file, header=True)

# Clean origins countries
for index, row in cleaned_countries.head(seen + interval).iterrows():
        if index < seen:
            continue
        if index%100 == 0:
            print(index)
        off_p_df = off_p_df.withColumn('origins', F.regexp_replace('countries', "^" + str(row['input']) + "$", row['country_code']))
        off_p_df = off_p_df.withColumn('manufacturing_places', F.regexp_replace('manufacturing_places', "^" + str(row['input']) + "$", row['country_code']))
        off_p_df = off_p_df.withColumn('countries', F.regexp_replace('countries', "^" + str(row['input']) + "$", row['country_code']))
        seen = index

# Save Spark dataframe in the Parquet format
# off_p_df.write.mode('overwrite').parquet(PARQUET_PATH)
off_p_df.toPandas().to_csv(temp_file, index=False)

print("Seen: {0}".format(seen))

7000
7100
Seen: 7150


In [11]:
off_p_df.show()

+-------------+--------------------+------------+-------------------------+--------------+---------+--------------------+-------+-----+
|         code|        product_name|origins_tags|manufacturing_places_tags|countries_tags|countries|manufacturing_places|origins|is_eu|
+-------------+--------------------+------------+-------------------------+--------------+---------+--------------------+-------+-----+
|0000000274722|Blanquette de Vol...|      france|                   france|        france|       ct|                  ct|     ct|    0|
|0000000290616|        Salade Cesar|      quebec|                   canada|        canada|       aq|                  aq|     aq|    0|
|0000000394710|Danoises à la can...|      quebec|                   canada|        canada|       aq|                  aq|     aq|    0|
|0000001071894|               Flute|      france|           united kingdom|united kingdom|       up|                  up|     up|    0|
|0000001938067|Chaussons tressés...|      quebec

In [ ]:
off_p_all_size = off_p_df.count()
off_p_cols_size = len(off_p_df.columns)
print("Full GEO information data Size:\n" + str(off_p_cols_size) + "(columns) * " + str(off_p_all_size) + "(rows)")

In [ ]:
# Detect european countries
eu_countries= pd.read_csv(DATA_FOLDER+ "/EU-countries.csv",sep=',', error_bad_lines=False, encoding = "utf-8")
eu_countries= eu_countries.Code
eu_countries.head()

In [ ]:
# store products that are sold in Switzerland
europe_sold_data = df[df["sold_in_EU"] == 1]

print("swiss_sold data Size:\n {0}".format(len(europe_sold_data)))

From now we work on products sold in Switzerland. 
We classify these products under different categories:
- products manufactured in Switzerland
- products not manufactured Switzerland to find which countries manufacture the most products consumed in Switzerland.
- products not originating from Switzerland to find which countries produce the ground ingredients for products sold in Switzerland.

In [ ]:
# store products which ingredients originate from Switzerland
swiss_origin_data = off_p_df.filter(off_p_df["origins"].rlike(filter_ch))

swiss_origin_size = swiss_origin_data.count()
print("swiss_origin data Size:\n" + str(swiss_origin_size))

In [ ]:
# store products that are manufactured in Switzerland
swiss_manu_data = off_p_df.filter(off_p_df["manufacturing_places"].rlike(filter_ch))

swiss_manu_size = swiss_manu_data.count()
print("swiss_manufactored data Size:\n" + str(swiss_manu_size))

In [ ]:
# store products that are both manufactured and originate from Switzerland
swiss_origin_manu_data = off_p_df.filter(off_p_df["origins"].rlike(filter_ch) & off_p_df["manufacturing_places"].rlike(filter_ch))

swiss_origin_manu_size = swiss_origin_manu_data.count()
print("swiss_origin_manufactored data Size:\n" + str(swiss_origin_manu_size))

In [ ]:
#count products whose ingredients from switzerland but which were not manufactured in Switzerland
swiss_origin_tot = swiss_origin_data.filter(~ swiss_origin_data["manufacturing_places"].rlike(filter_ch)).count()

#Total of products sold in Switzerland
swiss_sold_tot = swiss_sold_data.count()

#count products that do not have swiss ingredients
swiss_manu_tot = swiss_manu_data.filter(~ swiss_manu_data["origins"].rlike(filter_ch)).count()

#products that are sold in Switzerland but that were not manufactured in CH and that do not have swiss ingredients
non_swiss_sold_tot = swiss_sold_data.filter(~ swiss_sold_data["manufacturing_places"].rlike(filter_ch) & ~swiss_sold_data["origins"].rlike(filter_ch)).count()

#total of products that are both manufactured and has ingredients from Switzerland
swiss_origin_manu_tot = swiss_origin_manu_data.count()

labels = 'Swiss ingredients (but not manufactured in Switzerland)', 'Manufactured in Switzerland (with no swiss ingredients)', 'Completely foreign products', 'Both swiss ingredients and manufactured in Switzerland'
sizes = [swiss_origin_tot, swiss_manu_tot, non_swiss_sold_tot, swiss_origin_manu_tot]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Proportions for a total of ' + str(swiss_sold_tot) + ' products sold in Switzerland')

plt.show()

According to above chart, we can see that Switzerland tend to use local ingredients to manufacture products. Since most of products manufactured in Switzerland are produced by mostly Swiss ingredients.

In [ ]:
#Store products that are not manufactured in Switzerland
non_swiss_sold = swiss_sold_data.filter(~swiss_sold_data["manufacturing_places"].rlike(filter_ch))

non_swiss_sold_size = non_swiss_sold.count()
print("Non swiss/Sold in Switzerland data Size:\n" + str(non_swiss_sold_size))

In [ ]:
#Store products that are manufactured in Switzerland
swiss_m_sold = swiss_sold_data.filter(swiss_sold_data["manufacturing_places"].rlike(filter_ch))

swiss_m_sold_size = swiss_m_sold.count()
print("Swiss/Sold in Switzerland data Size:\n" + str(swiss_m_sold_size))

In [ ]:
# Plot the number Swiss vs non-swiss products (Manufacturing)
fig, ax = plt.subplots()
titles = ['Manufactured in Switzerland', 'Not manufactured in Switzerland']
plt.bar(titles, [swiss_m_sold_size, non_swiss_sold_size])
plt.ylabel('Number of products sold in Switzerland')   
plt.xticks(titles)
plt.title('Fig. Manufacturing products in Switzerland')
plt.show()

Out of __762__ products sold in Switzerland, Just about __51%__ of them was manufactured in Switzerland.

The dataset is not concistent with the country or places name, therefore, we have to clean the data by trying to unify a maximum the places names. First the database http://download.geonames.org/export/dump/ was used in order to match all the different ways of naming a country to a single convention but since there are some rows which need human intelligence to process(for example some fields contain name of ingredient or details) in the end we prefered to clean data manually. 

In [ ]:
cleaned_countries = pd.read_csv("cleaning_data/origins_cleaning.csv")

# Flatten origins column
origins_p = non_swiss_sold.withColumn('origins', F.explode_outer(F.split('origins', ',')))

# Clean origins countries
for index, row in cleaned_countries.iterrows():
    if(str(row['replace_with']) != "nan"):
        origins_p = origins_p.withColumn('origins', F.regexp_replace('origins', "^" + row['origins'] + "$", row['replace_with']))
    else:
        origins_p = origins_p.withColumn('origins', F.regexp_replace('origins', row['origins'], "Not Specified"))

In [ ]:
# Find dominant importers of ingredients

origins_p.createOrReplaceTempView("origins_p")
target_origins = spark.sql("SELECT origins, COUNT(origins) FROM origins_p GROUP BY origins ORDER BY COUNT(origins) DESC")
target_origins = target_origins.withColumnRenamed('count(origins)' , 'Count')

print("Number of ingredient importers:\n" + str(target_origins.count()))

In [ ]:
target_origins.show(5)

In [ ]:
# Extract the 10 most important countries and number of occurances in ingredients.
origin_countries = target_origins.toPandas() 
origin_countries = origin_countries.head(10)

# Plot the number of products imported by 10 most important countries
fig, ax = plt.subplots()
ax.grid(zorder=-1)
plt.bar(origin_countries.origins, origin_countries.Count, zorder=3, color='skyblue')
plt.ylabel('Number of products')   
plt.xticks(origin_countries.origins, rotation='80')
plt.title('Fig. The number of products which ingredients are from the 10 most frequent countries')
plt.show()

In [ ]:
# Export origins to CSV 
target_origins.select("origins").toPandas().to_csv('output/origins.csv')

Now that we found the most important countries in case of producing ingredients for products which are imported to Switzerland, We should take the same approach to find the most important manufacturers of this products.

In [ ]:
# Flatten manufacturing_places column
manufacturers_p = non_swiss_sold.withColumn('manufacturing_places', F.explode_outer(F.split('manufacturing_places', ',')))

# Clean manufacturers countries
for index, row in cleaned_countries.iterrows():
    if(str(row['replace_with']) != "nan"):
        manufacturers_p = manufacturers_p.withColumn('manufacturing_places', F.regexp_replace('manufacturing_places', "^" + row['origins'] + "$", row['replace_with']))
    else:
        manufacturers_p = manufacturers_p.withColumn('manufacturing_places', F.regexp_replace('manufacturing_places', row['origins'], "Not Specified"))


In [ ]:
# Find dominant importers (manufacturers)

manufacturers_p.createOrReplaceTempView("manufacturers_p")
target_manufacturers = spark.sql("SELECT manufacturing_places, COUNT(manufacturing_places) FROM manufacturers_p GROUP BY manufacturing_places ORDER BY COUNT(manufacturing_places) DESC")
target_manufacturers = target_manufacturers.withColumnRenamed('count(manufacturing_places)' , 'Count')
# target_manufacturers.show()

print("Number of Manufacturers:\n" + str(target_manufacturers.count()))

In [ ]:
# Extract the 10 most important manufacturers countries and number of occurances.
manufacturer_countries = target_manufacturers.toPandas() 
manufacturer_countries = manufacturer_countries.head(10)

# Plot the number of products manufactured by 10 most important countries
fig, ax = plt.subplots()
ax.grid(zorder=-1)
plt.bar(manufacturer_countries.manufacturing_places, manufacturer_countries.Count, zorder=3, color='skyblue')
plt.ylabel('Number of products')   
plt.xticks(manufacturer_countries.manufacturing_places, rotation='80')
plt.title('Fig. The number of products manufactured by 10 most important countries')
plt.show()

In [ ]:
# Export manufacturing_places to CSV 
target_manufacturers.select("manufacturing_places").toPandas().to_csv('output/manufacturing_places.csv')

### Working with products' categories

We would like to find which countries import what products?
In order to answer to this question, first we extract categories corresponding to the products sold in Switzerland but not manufactured in this country. Then we will combine the information provided for manufacturing places of these products with corresponding category. 

In the end we will extract the 5 most important importers of each category.

In [ ]:
extra_info_df = ' categories '
categories_df = spark.sql("SELECT" + p_id_col + extra_info_df + " FROM data_main")
categories_df.show(10)

In [ ]:
categories_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in traces_df.columns]).show()

In [ ]:
# Join table of categories with table of target products (Sold in Switzerland but not manufactured in it)
non_swiss_sold.createOrReplaceTempView("target_products_df")
categories_df.createOrReplaceTempView("categories_df")
joined_df = spark.sql("SELECT p.code, c.categories, p.origins, p.manufacturing_places, p.countries  FROM target_products_df p INNER JOIN categories_df c ON p.code = c.code")

In [ ]:
joined_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in joined_df.columns]).show()

According to above table, just 15 products do not have category. 

In [ ]:
joined_df.createOrReplaceTempView("target_products_cats")
sql_filter = "SELECT * FROM target_products_cats WHERE categories is not NULL"
target_products_categories_p = spark.sql(sql_filter)

print("Number of Products sold in Switzerland with categories:\n" + str(target_products_categories_p.count()))

In [ ]:
target_products_categories_p.show(5)

In [ ]:
# Flatten categories column
target_products_categories_p = target_products_categories_p.withColumn('categories', F.explode_outer(F.split('categories', ',')))

# Remove occurances of en: in name of categories
target_products_categories_p = target_products_categories_p.withColumn('categories', F.regexp_replace('categories', 'en:', ''))

target_products_categories_p.show(5)

In [ ]:
# Find dominant categories
target_products_categories_p.createOrReplaceTempView("target_products_categories_p")
target_categories = spark.sql("SELECT categories, COUNT(categories) FROM target_products_categories_p GROUP BY categories ORDER BY COUNT(categories) DESC")
target_categories = target_categories.withColumnRenamed('count(categories)' , 'Count')
target_categories.show()

print("Number of Categories:\n" + str(target_categories.count()))

According to above table not all categories are independant and they may contain similar products. We did not change categories for this milestone but we would like to merge some categories in future to have better distinction between countries which import these products. 

In [ ]:
# Export Categories to CSV 
target_categories.toPandas().to_csv('output/categories.csv')

In [ ]:
# Extract the 15 most frequent categories and number of occurances.
categories = target_categories.toPandas() 
categories = categories.head(15)

# Plot the most frequent categories
fig, ax = plt.subplots()
ax.grid(zorder=-1)
plt.barh(categories.categories, categories.Count, zorder=3, color='skyblue')
plt.xlabel('Number of products')   
plt.title('Fig. The number of products sold in Switzerland')
ax.invert_yaxis() 
plt.show()

In [ ]:
cleaned_countries = pd.read_csv("cleaning_data/origins_cleaning.csv")

In [ ]:
# Extract the most important manufacturer countries for 15 most frequent categories

categories_countires_p = target_products_categories_p.withColumn('manufacturing_places', F.explode_outer(F.split('manufacturing_places', ',')))

# Clean manufacturers countries
for index, row in cleaned_countries.iterrows():
    if(str(row['replace_with']) != "nan"):
        categories_countires_p = categories_countires_p.withColumn('manufacturing_places', F.regexp_replace('manufacturing_places', "^" + row['origins'] + "$", row['replace_with']))
    else:
        categories_countires_p = categories_countires_p.withColumn('manufacturing_places', F.regexp_replace('manufacturing_places', row['origins'], "Not Specified"))

categories_countires_p = categories_countires_p.withColumn('categories', F.explode_outer(F.split('categories', ',')))
products_cats_df = categories_countires_p.toPandas()

countries_cats = []
cat = "Snacks sucrés"
for index, row in categories.iterrows():
    temp_df = products_cats_df[(products_cats_df.categories == row.categories)].groupby('manufacturing_places').count().sort_values(by=['code'], ascending=False).head(5)
    countries_cats.append(temp_df)

# Each element of countries_cats is a dataframe containing the most important manufacturers
# of the corresponding category

i = 0
print("category: " + str(categories.iloc[i].categories))
print("The most important manufacturers:")
print(countries_cats[0].head(3).index.values)

We will use extracted information about the most important manufacturer countries for 15 most frequent categories for our visualization.

## Descriptive Analysis

In [ ]:
openfood = pd.read_csv(DATA_FOLDER + openfood_file, sep = '\t')

In [ ]:
foodSwitzerland = openfood[openfood['countries_tags']=="en:switzerland"]

In [ ]:
nbProdSwit = len(foodSwitzerland)
print("Number of products: ", nbProdSwit)

# first perspective: Compare old vs new products inside the dataset

A product in the dataset is considered "old" if its description was uploaded to the dataset before February 2017. By the contrary, it is considered as "new" if its description was uploaded after that date.

_As that definition is a little rigid, in order to get closer to the real situation, the following assumption is made: From the total products uploaded to the dataset after February 2017, 20% are old products._  

A first perspective to tackle the research question, is to compare how in this more than 6 years of existance of the dataset, the characteristics of the products have changed. Specifically, we would like to know if there have been some changes in the origin of the primary resources, or in the origin of the manufacture or in the labels of the products.

The study of the evolution in time of each one of those features, will include an __exploratory data analysis__.
Finally, a study including the three features will be done, aiming to find an aggregated differentiated behavior in time, reflected in different clusters of periods of time. For that the __K-modes algorithm__ will be used.

In [ ]:
print("Date of first upload: ", min(foodSwitzerland['created_datetime']))
print("Date of last upload retrieved: ", max(foodSwitzerland['created_datetime']))

In [ ]:
foodSwitzerland['created_datetime'] = pd.to_datetime(foodSwitzerland['created_datetime'])
foodSwitzerland = foodSwitzerland.sort_values(by='created_datetime')

The uploads of products sold in Switzerland, behaves differently in time. Taking into account the histogram presented below, two periods of time are defined:
- Period 1: Created for studying the behaviour from "old" products. Products uploaded before Feb 2017.
- Period 2: Created for studying the behaviour from "new" products. Products uploaded after Feb 2017.

In [ ]:
rateOldInNew = 0.2
old_products = len(foodSwitzerland[foodSwitzerland['created_datetime']<"2017-03-01 00:00:00"])
print("Old products: ", old_products + (len(foodSwitzerland)-old_products)*(rateOldInNew))
print("New products: ", (len(foodSwitzerland)-old_products)*(1-rateOldInNew))
foodSwitzerland['created_datetime'].hist()

Before making subdivision of data:

In [ ]:
filter_ch = '[Ss]witzerland|[Ss]uisse|[Ss]chweiz|[Ss]vizerra'
filter_local = '[Ss]witzerland|[Ss]uisse|[Ss]chweiz|[Ss]vizerra|[Ll]ocal'

place = pd.Series(['Other country','Switzerland', 'No information'], index=[0,1,2])
refLabel = pd.Series(['Other Label','Related with Switzerland', 'No information'], index=[0,1,2]) 

foodSwitzerland["originsCat"] = foodSwitzerland["origins"].str.contains(filter_ch,regex=True).map(place,na_action='ignore')
foodSwitzerland["manuCat"] = foodSwitzerland["manufacturing_places"].str.contains(filter_ch,regex=True).map(place,na_action='ignore')
foodSwitzerland["labCat"] = foodSwitzerland["labels_tags"].str.contains(filter_local,regex=True).map(refLabel,na_action='ignore')

Now, let's do the subdivision in the two periods, taking care of our assumption of 20%:

In [ ]:
foodSwitzerlandBef = foodSwitzerland[foodSwitzerland['created_datetime']<"2017-03-01 00:00:00"]
foodSwitzerlandAft = foodSwitzerland[foodSwitzerland['created_datetime']>="2017-03-01 00:00:00"]

befInAft = foodSwitzerlandAft.sample(n=int(rateOldInNew*len(foodSwitzerlandAft)), replace=False)
foodSwitzerlandBef = pd.concat([foodSwitzerlandBef,befInAft],axis=0)

for i in range (0,len(befInAft)):
    foodSwitzerlandAft = foodSwitzerlandAft[foodSwitzerlandAft['code']!=befInAft['code'].iloc[i]]
print("Number of Old products: ", np.shape(foodSwitzerlandBef))
print("Number of New products: ", np.shape(foodSwitzerlandAft))

### 2.1 Study of the evolution in time of each one of the interest features

### 2.1.1 With respect to: Origin of the primary resources

First, bootstrapping for include confidence intervals to results

In [ ]:
#before
propSB_100ite = np.zeros (100)
propOCB_100ite = np.zeros (100)
#after
propSA_100ite = np.zeros (100)
propOCA_100ite = np.zeros (100)

for iteration in range(0,100):
    #before
    temp_bef = foodSwitzerlandBef["originsCat"].sample(n=len(foodSwitzerlandBef), replace=True)
    propSB_100ite[iteration] = (sum(temp_bef=="Switzerland")/len(foodSwitzerlandBef))
    propOCB_100ite[iteration] = sum(temp_bef=="Other country")/len(foodSwitzerlandBef)
    #after
    temp_aft = foodSwitzerlandAft["originsCat"].sample(n=len(foodSwitzerlandAft), replace=True)
    propSA_100ite[iteration] = sum(temp_aft=="Switzerland")/len(foodSwitzerlandAft)
    propOCA_100ite[iteration] = sum(temp_aft=="Other country")/len(foodSwitzerlandAft)
    
    #Relation of time evolution and difference of products in each categories
    difpropB = propSB_100ite[iteration]-propOCB_100ite[iteration]
    difpropA = propSB_100ite[iteration]-propOCA_100ite[iteration]

#Calculating standard deviation of count of each category
SBstd = np.std(propSB_100ite)
OCBstd = np.std(propOCB_100ite)
#Calculating standard deviation of count of each category
SAstd = propSA_100ite.std()
OCAstd = propOCA_100ite.std()

Then, plotting of behavior 

In [ ]:
#Report information of NAN cases
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
print("There was not information for ",len(foodSwitzerlandBef["originsCat"])-sum(foodSwitzerlandBef["originsCat"]=="Switzerland")-sum(foodSwitzerlandBef["originsCat"]=="Other country")," products in the period 1.")
print("There was not information for ",len(foodSwitzerlandAft["originsCat"])-sum(foodSwitzerlandAft["originsCat"]=="Switzerland")-sum(foodSwitzerlandAft["originsCat"]=="Other country")," products in the period 2.")

#Plot origin of primary resources by periods
plt.subplot(1,2,1)
(foodSwitzerlandBef["originsCat"].value_counts()/len(foodSwitzerlandBef["originsCat"])).plot(kind='bar', yerr = [SBstd,OCBstd],title='Period 1')
plt.subplot(1,2,2)
(foodSwitzerlandAft["originsCat"].value_counts()/len(foodSwitzerlandAft["originsCat"])).plot(kind='bar', yerr = [SAstd,OCAstd],title='Period 2')

plt.show()

Most of old products have as origin of primary resources Switzerland. However, in new products, the origin of primary resources is for Switzerland and Other countries statistically the same.

### 2.1.2 With respect to: Manufacture

First, bootstrapping for include confidence intervals to results

In [ ]:
#before
propSB_100ite = np.zeros (100)
propOCB_100ite = np.zeros (100)

#after
propSA_100ite = np.zeros (100)
propOCA_100ite = np.zeros (100)

for iteration in range(0,100):
    #before
    temp_bef = foodSwitzerlandBef["manuCat"].sample(n=len(foodSwitzerlandBef), replace=True)
    propSB_100ite[iteration] = sum(temp_bef=="Switzerland")/len(foodSwitzerlandBef)
    propOCB_100ite[iteration] = sum(temp_bef=="Other country")/len(foodSwitzerlandBef)
    #after
    temp_aft = foodSwitzerlandAft["manuCat"].sample(n=len(foodSwitzerlandAft), replace=True)
    propSA_100ite[iteration] = sum(temp_aft=="Switzerland")/len(foodSwitzerlandAft)
    propOCA_100ite[iteration] = sum(temp_aft=="Other country")/len(foodSwitzerlandAft)
    

#Calculating standard deviation of count of each category
SBstd = propSB_100ite.std()
OCBstd = propOCB_100ite.std()
#Calculating standard deviation of count of each category
SAstd = propSA_100ite.std()
OCAstd = propOCA_100ite.std()

In [ ]:
#Report information of NAN cases
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
print("There was not information for ",len(foodSwitzerlandBef["manuCat"])-sum(foodSwitzerlandBef["manuCat"]=="Switzerland")-sum(foodSwitzerlandBef["manuCat"]=="Other country")," products in the period 1.")
print("There was not information for ",len(foodSwitzerlandAft["manuCat"])-sum(foodSwitzerlandAft["manuCat"]=="Switzerland")-sum(foodSwitzerlandAft["manuCat"]=="Other country")," products in the period 2.")

#Plot origin of primary resources by periods
plt.subplot(1,2,1)
(foodSwitzerlandBef["manuCat"].value_counts()/len(foodSwitzerlandBef["manuCat"])).plot(kind='bar', yerr = [SBstd,OCBstd],title='Period 1')
plt.subplot(1,2,2)
(foodSwitzerlandAft["manuCat"].value_counts()/len(foodSwitzerlandBef["manuCat"])).plot(kind='bar', yerr = [SAstd,OCAstd],title='Period 2')

plt.show()

Most of old products were manufactured in Switzerland. However, in new products, the manufacture is for Switzerland and Other countries statistically the same.

### 2.1.3 With respect to: Labels

In [ ]:
#before
propSB_100ite = np.zeros (100)
propOCB_100ite = np.zeros (100)
#after
propSA_100ite = np.zeros (100)
propOCA_100ite = np.zeros (100)

for iteration in range(0,100):
    #before
    temp_bef = foodSwitzerlandBef["labCat"].sample(n=len(foodSwitzerlandBef), replace=True)
    propSB_100ite[iteration] = sum(temp_bef=="Related with Switzerland")/len(foodSwitzerlandBef)
    propOCB_100ite[iteration] = sum(temp_bef=="Other Label")/len(foodSwitzerlandBef)
    #after
    temp_aft = foodSwitzerlandAft["labCat"].sample(n=len(foodSwitzerlandAft), replace=True)
    propSA_100ite[iteration] = sum(temp_aft=="Related with Switzerland")/len(foodSwitzerlandAft)
    propOCA_100ite[iteration] = sum(temp_aft=="Other Label")/len(foodSwitzerlandAft)
    
    #Relation of time evolution and difference of products in each categories
    difpropB = propSB_100ite[iteration]-propOCB_100ite[iteration]
    difpropA = propSB_100ite[iteration]-propOCA_100ite[iteration]

#Calculating standard deviation of count of each category
SBstd = propSB_100ite.std()
OCBstd = propOCB_100ite.std()
#Calculating standard deviation of count of each category
SAstd = propSA_100ite.std()
OCAstd = propOCA_100ite.std()

In [ ]:
#Report information of NAN cases
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
print("There was not information for ",len(foodSwitzerlandBef["labCat"])-sum(foodSwitzerlandBef["labCat"]=="Switzerland")-sum(foodSwitzerlandBef["labCat"]=="Other country")," products in the period 1.")
print("There was not information for ",len(foodSwitzerlandAft["labCat"])-sum(foodSwitzerlandAft["labCat"]=="Switzerland")-sum(foodSwitzerlandAft["labCat"]=="Other country")," products in the period 2.")

#Plot origin of primary resources by periods
plt.subplot(1,2,1)
(foodSwitzerlandBef["labCat"].value_counts()/len(foodSwitzerlandBef["labCat"])).plot(kind='bar', yerr = [SBstd,OCBstd],title='Period 1')
plt.subplot(1,2,2)
(foodSwitzerlandAft["labCat"].value_counts()/len(foodSwitzerlandBef["labCat"])).plot(kind='bar', yerr = [SAstd,OCAstd],title='Period 2')

plt.show()

For the two periods studied, other labels are more frequent than the related with Switzerland

## Future plan

### Improving data cleaning for countries 

Although we cleaned the data of name of countries but about 200 are still remained.

### Extend research questions to Europe

After talking with TA, we decided to extend our research questions to Europe in order to have more data. The similar approach will be taken for answering our main questions about Europe.

### Evaluate and complement our results

We would like to complement our results using following databases:
- Additional datasets “Evolution de la consommation de denrées alimentaires en Suisse” (https://opendata.swiss/fr/dataset/entwicklung-des-nahrungsmittelverbrauches-in-der-schweiz-je-kopf-und-jahr1) and “Dépenses fédérales pour l’agriculture et l’alimentation” (https://opendata.swiss/fr/dataset/bundesausgaben-fur-die-landwirtschaft-und-die-ernahrung1) from https://opendata.swiss/fr/group/agriculture

- A last additional dataset for the second question of the project https://www.gate.ezv.admin.ch/swissimpex/public/bereiche/waren/result.xhtml Total of imports of agriculture, forestry and fishing goods

### Find the most important characteristic of Swiss-made products

After answering our main research questions we would like to find which products are likely to import to Switzerland. To answer this question we would like to train a classifier in order to see which features are the most important features to decide if a product is Swiss made or not.

### Improve our assumption about date of entering products to the Swiss market

Study of the evolution in time of the interest features combined
- The k-modes and other machine learning algorithms are expected to be done for the milestone 3.